
### This notebook will;
#### 1. Read the Delta Lake ingested files and perform transformations - final data to be analyzed using Azure Synapse
#### 2. Save transformed data into 'ADLS Transformation Layer', again with storage format as Delta Lake & partitioned by date

Note: The %run command is used to get variables/ functions that are common for both ingest and transform notebooks

In [ ]:
%run "./FlightData-commonconfigs"

In [ ]:
####################################### TRANSFORMATION #######################################

def transformation(storage_account_name,storage_container_name):

    # Defining the schema of the dataframe
    schema = StructType([
            StructField("scheduled_time", IntegerType(), True), 
            StructField("flight_no", StringType(), True),
            StructField("airport_city", StringType(), True),
            StructField("airport_code", StringType(), True),
            StructField("airline", StringType(), True),
            StructField("aircraft_type", StringType(), True),
            StructField("aircraft_registration", StringType(), True),
            StructField("actual_time", IntegerType(), True)
    ])

    # Read delta file, partitioned by date, from ingestion ADLS layer
    df = spark.read.format("delta").load(f"/mnt/{storage_account}/{storage_ingestion_container}/{date.today() - timedelta(days=1)}")

    # Drop rows if scheduled/ actual times or airline are NULL i.e. unknown
    df = df.filter(df["scheduled_time"].isNotNull())
    df = df.filter(df["actual_time"].isNotNull())
    df = df.filter(df["airline"].isNotNull())

    # Getting time difference between actual and scheduled
    # Delay in seconds
    df = df.withColumn("delay", df["actual_time"] - df["scheduled_time"])

    # Convert epoch to timestamp
    df = df.withColumn("scheduled_time", F.to_timestamp(df["scheduled_time"]))
    df = df.withColumn("actual_time", F.to_timestamp(df["actual_time"]))

    # Convert timestamp to appropriate timezone - "Europe/Malta"
    df = df.withColumn("scheduled_time", F.from_utc_timestamp(df["scheduled_time"],"Europe/Malta"))
    df = df.withColumn("actual_time", F.from_utc_timestamp(df["actual_time"],"Europe/Malta"))

    # Sort by scheduled time parameter
    df = df.sort(df["scheduled_time"])

    # Mount transformation container
    mount_adls_using_sp(storage_account,storage_transformation_container)

    # Write dataframe to ADLS (as Delta Lake) in 'ADLS Transformation Layer', again partitioned by date
    df.write.format("delta").save(f"/mnt/{storage_account_name}/{storage_container_name}/{date.today() - timedelta(days=1)}")

In [ ]:
# Driver code

####################################### TRANSFORMATION #######################################

# Transform data and write to 'ADLS Transformation Layer'
transformation(storage_account,storage_transformation_container)

In [ ]:
# TEST

display(spark.read.format("delta").load(f"/mnt/{storage_account}/{storage_transformation_container}/{date.today() - timedelta(days=1)}"))